## Loading files into mongodb

In [1]:
from pymongo import MongoClient # Install for using this lib.
import subprocess

In [2]:
dbname = 'test'
collection_name1 = 'reviews'
collection_name2 = 'business'
collection_name3 = 'users'

path = "/Users/alvira/Desktop/distributedData/project/"

input_file_name1 = path+"review.json"
input_file_name2 = path+'business.json'
input_file_name3 = path+'user.json'



In [3]:
def import_query(dbname, collection_name, input_file_name):
    mongoimport_query =  'mongoimport --db ' + dbname + ' --collection ' + collection_name + ' --file ' + input_file_name
    #COMPLETE THIS.
    return mongoimport_query
 
#Create connection 
client = MongoClient() #default-localhost:27017
#Connect to database
db = client[dbname]

#Drop table.
db[collection_name1].drop()
db[collection_name2].drop()
db[collection_name3].drop()

#Insert all data from the input_file_name.

In [4]:
mongoimport_query = import_query(dbname, collection_name1, input_file_name1)
subprocess.call(mongoimport_query,shell=True)

mongoimport_query = import_query(dbname, collection_name2, input_file_name2)
subprocess.call(mongoimport_query,shell=True)

mongoimport_query = import_query(dbname, collection_name3, input_file_name3)
subprocess.call(mongoimport_query,shell=True)

0

## Keep only restaurants

In [5]:
restaurants = db.business.find({ '$or' : [ { "attributes.RestaurantsReservations" : {'$exists' : 'true'} }, { "attributes.RestaurantsDelivery" : {'$exists':'true'}}, 
                            { "attributes.RestaurantsAttire" : {'$exists':'true'}}, { "attributes.RestaurantsCounterService" : {'$exists':'true'}}, 
                            { "attributes.RestaurantsGoodForGroups" : {'$exists':'true'}}, { "attributes.RestaurantsPriceRange2" : {'$exists':'true'}}, 
                            { "attributes.RestaurantsTableService" : {'$exists':'true'}}, { "attributes.RestaurantsTakeOut" : {'$exists':'true'}}, 
                            { "attributes.GoodForMeal" : {'$exists':'true'}}]})

We'll have to rewrite the restaurants into mongo db or so, or just filter the business db to contain only those that are in restaurants. I've just created a filtered list for now.

In [6]:
len(list(restaurants))

99566

#### Finding restuarants having review counts greater than 25

In [34]:
restaurants = db.business.find({ '$or' : [ { "attributes.RestaurantsReservations" : {'$exists' : 'true'} }, { "attributes.RestaurantsDelivery" : {'$exists':'true'}}, 
                            { "attributes.RestaurantsAttire" : {'$exists':'true'}}, { "attributes.RestaurantsCounterService" : {'$exists':'true'}}, 
                            { "attributes.RestaurantsGoodForGroups" : {'$exists':'true'}}, { "attributes.RestaurantsPriceRange2" : {'$exists':'true'}}, 
                            { "attributes.RestaurantsTableService" : {'$exists':'true'}}, { "attributes.RestaurantsTakeOut" : {'$exists':'true'}}, 
                            { "attributes.GoodForMeal" : {'$exists':'true'}}], 
                    "review_count":{"$gt": 25}})

In [35]:
rest = list(restaurants)

In [ ]:
# Collecting business ID's
business_ids = []
for i in range(len(rest)):
    business_ids.append(rest[i]['business_id'])

In [68]:
len(business_ids)

30152

In [ ]:
# list(db.business.find({ '$or' : [ { "attributes.RestaurantsReservations" : {'$exists' : 'true'} }, { "attributes.RestaurantsDelivery" : {'$exists':'true'}}, 
#                             { "attributes.RestaurantsAttire" : {'$exists':'true'}}, { "attributes.RestaurantsCounterService" : {'$exists':'true'}}, 
#                             { "attributes.RestaurantsGoodForGroups" : {'$exists':'true'}}, { "attributes.RestaurantsPriceRange2" : {'$exists':'true'}}, 
#                             { "attributes.RestaurantsTableService" : {'$exists':'true'}}, { "attributes.RestaurantsTakeOut" : {'$exists':'true'}}, 
#                             { "attributes.GoodForMeal" : {'$exists':'true'}}]}))

#### Filtering users (having reviews count greater than 25)

In [64]:
filtered_users = db.users.find({"review_count":{"$gt": 25}})

In [65]:
user = list(filtered_users )

In [66]:
user_ids = []
for i in range(len(user)):
    user_ids.append(user[i]['user_id'])

In [67]:
len(user_ids)

203189

#### Filtering reviews basis above user_id & business_id and stars

In [76]:
len(list(db.reviews.find()))

4736897

In [84]:
filtered_reviews = db.reviews.find({"user_id": {'$in':user_ids}, "business_id": {'$in':business_ids}, 
                                    "stars": {'$in':[1,2,3,4,5]}})

In [85]:
review = list(filtered_reviews )

In [87]:
len(review)

1727718

In [86]:
db.restaurant.drop()

1727718

Find a way to save restaurants as a table. This doesnt work:

In [ ]:
db.restaurant.insert(restaurants)

## Load tables into pyspark

In [6]:
reviews = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://127.0.0.1/test.reviews").load()

# Replace with only restaurants
business = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://127.0.0.1/test.business").load()

users = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://127.0.0.1/test.users").load()

In [10]:
reviews.show(1)

+--------------------+--------------------+----+----------+-----+--------------------+-----+--------------------+------+--------------------+
|                 _id|         business_id|cool|      date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+--------------------+----+----------+-----+--------------------+-----+--------------------+------+--------------------+
|[5a5a5e8eda8b9b57...|uYHaNptLzDLoV_JZ_...|   0|2016-07-12|    0|VfBHSwC5Vz_pbFluy...|    5|My girlfriend and...|     0|cjpdDjZyprfyDG3Rl...|
+--------------------+--------------------+----+----------+-----+--------------------+-----+--------------------+------+--------------------+
only showing top 1 row



In [13]:
business.show(10)

+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+-------+----------+------------+--------------------+-------------+-----------+------------+-----+-----+
|                 _id|             address|          attributes|         business_id|          categories|            city|               hours|is_open|  latitude|   longitude|                name| neighborhood|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+--------------------+-------+----------+------------+--------------------+-------------+-----------+------------+-----+-----+
|[5a5a5f23da8b9b57...|7702 E Doubletree...|[null,null,null,n...|CVtCbSB1zUcUWg-9T...|[Professional Ser...|      Scottsdale|[9:00-17:00,9:00-...|      1|33.5650816|-111.9164003|            TRUmatch|             |      85258|           3|  3.0|   AZ|
|[5a

In [12]:
users.show(1)

+--------------------+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+-----+--------------------+----+--------------------+-----+----+------------+------+--------------------+-------------+
|                 _id|average_stars|compliment_cool|compliment_cute|compliment_funny|compliment_hot|compliment_list|compliment_more|compliment_note|compliment_photos|compliment_plain|compliment_profile|compliment_writer| cool|               elite|fans|             friends|funny|name|review_count|useful|             user_id|yelping_since|
+--------------------+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+-----+--------------------+----+--------------------+-----+----+------------+------+----------------

In [14]:
business.select('attributes')

DataFrame[attributes: struct<AcceptsInsurance:boolean,AgesAllowed:string,Alcohol:string,Ambience:struct<casual:boolean,classy:boolean,divey:boolean,hipster:boolean,intimate:boolean,romantic:boolean,touristy:boolean,trendy:boolean,upscale:boolean>,BYOB:boolean,BYOBCorkage:string,BestNights:struct<monday:boolean,tuesday:boolean,friday:boolean,wednesday:boolean,thursday:boolean,sunday:boolean,saturday:boolean>,BikeParking:boolean,BusinessAcceptsBitcoin:boolean,BusinessAcceptsCreditCards:boolean,BusinessParking:struct<garage:boolean,lot:boolean,street:boolean,valet:boolean,validated:boolean>,ByAppointmentOnly:boolean,Caters:boolean,CoatCheck:boolean,Corkage:boolean,DogsAllowed:boolean,DriveThru:boolean,GoodForDancing:boolean,GoodForKids:boolean,GoodForMeal:struct<dessert:boolean,latenight:boolean,lunch:boolean,dinner:boolean,breakfast:boolean,brunch:boolean>,HairSpecializesIn:struct<africanamerican:boolean,asian:boolean,coloring:boolean,curly:boolean,extensions:boolean,kids:boolean,perms:b